# YAML Joiner

## Aims

The aim of this small module is to merge two exported [anaconda](https://www.anaconda.com/) environment files (.yml or .yaml).

There is no easy way to merge the two files without duplicating some entries. 

In [1]:
# Import the modules

import os

In [2]:
# Declare variables, the yaml files to merge 

file_master = "src/proto_bal.yaml"
file_slave = "src/STEPv1.yaml"
file_target = "proto_bal.yaml"

In [3]:
def import_file(file):
    """ returns the parts of the files to be duplicated:
        - dependencies
        - its pip subparts
        
        Args: the file name
        
        Returns:
        - a list of the dependencies
        - a list of the pip entries
    """
    
    # 1. Import the file as a list of lines
    with open(file) as f:
        lines = f.readlines()
    
    # 2. declare the variables    
    dependencies = []
    pip = []
    write_dep = False
    write_pip = False
    
    # 3. loop the list to be searched 
    for line in lines:
        if write_dep:
            dependencies.append(line)
        if write_pip:
            pip.append(line)
        
        # 3.1. Enable the addition of dependencies
        #      when the header is found
        if "dependencies" in line:
            write_dep = True
        
        # 3.2. Enable the addition of pip
        #      when the header is found and close 
        #      the dependencies writing
        if "pip:" in line:
            write_pip = True
            write_dep = False
            
        # 3.3. Break the list when prefix is call
        if "prefix:" in line:
            break
    
    # 4. Clean up the list: they kicking the last entry
    #    which is always the following header
    
    dependencies = dependencies[:-1]
    pip = pip[:-1]
    
    return dependencies, pip

In [4]:
# Fetch the data:
# 
slave_dep, slave_pip = import_file(file_slave)
master_dep, master_pip = import_file(file_master)

In [5]:
def add_master(masters, slaves):
    """ extracts the missing entries out of slaves
        
        Args:
        - master, the file where the results should be merged to
        - slave, the file from where the additions come

        Returns:
        - The differences
    """
    output = []
    for slave in slaves:
        
        slave_name = slave[:slave.index("=")]
        counter = 0
        
        for master in masters:
            if slave_name in master:
                counter +=1
                break
        
        if counter == 0:
            output.append(slave)
            
    return output

In [6]:
# Extract the difference from the dependencies
# and join it to the master values

dep_add = add_master(master_dep, slave_dep)
master_dep += dep_add

print "Adding %s dependencies" %len(dep_add)
# Extract the difference from the pip entries
# and join it to the master values

pip_add = add_master(master_pip, slave_pip)
master_pip += pip_add

print "Adding %s pip entries" %len(pip_add)

Adding 11 dependencies
Adding 7 pip entries


In [7]:
def import_headers(file):
    """ Returns the header and closing statements
        which remains constant
    """
    with open(file) as f:
        lines = f.readlines()
        
    header = []

    for line in lines:
        header.append(line)
        
        if "dependencies" in line:
            break
    
    closing = lines[-2:-1]
    
    return header, closing

In [8]:
master_head, master_closing = import_headers(file_master)

In [9]:
new_yaml = master_head + master_dep + ['- pip:\n'] + master_pip + master_closing

In [10]:
def write_to_file(filename, list_lines, path = 'dir/'):
    """ writes the list to a file
    
        Args:
        - filename, as string
        - list_lines, the list we produced
        - path, as a string
    """
    with open(path + filename, "w") as f:
        for line in list_lines:
            f.write(line)
            
    return

In [11]:
write_to_file(file_target, new_yaml)